
# Proyecto 1 — Matematicas Computacionales

### Juan Rojas - Camilo Caballero

---

Este cuaderno implementa un **motor aritmetico de 16 bits en complemento a 2** (listas de bits) con: **suma, resta, multiplicacion y division** de enteros (positivos y negativos), **sin parentesis** y **con precedencia** `*` y `/` antes que `+` y `-`.

Incluye:
- Conversion `int <-> bits` (16 bits, C2) sin librerias externas.
- Operadores a nivel de bits: `add`, `sub`, `mul`, `div` con envoltura modulo `2^16`.
- Tokenizador, Shunting Yard y evaluacion en RPN (postfijo).
- **UI en Tkinter**


## Nucleo de 16 bits

In [3]:
# Juan: Motor aritmetico de 16 bits

BIT_WIDTH = 16  # Juan: Se instala el ancho de palabra en 16 bits (cantidad de bits usados)

def _mask16(n: int) -> int:

    # Juan: Se aplica mascara de 16 bits para mantener solo los bits 0..15 del numero

    return n & 0xFFFF  # Juan: Se hace AND con 0xFFFF (binario 16 unos) para recortar a 16 bits

def int_to_bits(n: int):

    # Juan: Se convierte un entero Python en lista de 16 bits (MSB a la izquierda) en representacion C2

    n = _mask16(n)  # Juan: Se asegura que el entero quede limitado a 16 bits aplicando mascara

    return [(n >> (BIT_WIDTH - 1 - i)) & 1 for i in range(BIT_WIDTH)]

    # Juan: Se recorre de MSB a LSB y se extrae cada bit usando corrimiento y mascara 1

def bits_to_int(bits):

    # Juan: Se convierte una lista de 16 bits (MSB a la izquierda) en un entero con signo en C2

    assert len(bits) == BIT_WIDTH  # Juan: Se valida que haya exactamente 16 bits

    unsigned = 0  # Juan: Se instala acumulador para valor sin signo

    for b in bits:  # Juan: Se recorre cada bit de izquierda a derecha

        unsigned = (unsigned << 1) | (b & 1)  # Juan: Se desplaza y agrega el bit actual

    if bits[0] == 1:  # Juan: Si el MSB es 1, el numero es negativo en C2

        return unsigned - (1 << BIT_WIDTH)  # Juan: Se resta 2^16 para obtener valor con signo

    return unsigned  # Juan: Si el MSB es 0, se retorna como positivo

def is_negative(bits):

    # Juan: Se retorna True si el bit de signo (MSB) es 1

    return bits[0] == 1

def copy_bits(bits):

    # Juan: Se crea una copia nueva de la lista de bits

    return bits[:]

def add_bits(a, b):

    # Juan: Se realiza suma de 16 bits en C2 con acarreo. El overflow se envuelve automaticamente

    assert len(a) == BIT_WIDTH and len(b) == BIT_WIDTH  # Juan: Se verifica que ambos tengan 16 bits

    res = [0] * BIT_WIDTH  # Juan: Se instala lista vacia para el resultado de 16 bits

    carry = 0  # Juan: Se instala variable para el acarreo inicial

    for i in range(BIT_WIDTH - 1, -1, -1):  # Juan: Se recorre de LSB a MSB

        total = a[i] + b[i] + carry  # Juan: Se suma bit a bit incluyendo el acarreo

        res[i] = total & 1  # Juan: Se guarda el bit menos significativo del resultado

        carry = (total >> 1) & 1  # Juan: Se actualiza el acarreo (bit mas significativo)

    return res  # Juan: Se retorna la suma final

def invert_bits(a):
    # Juan: Se invierten todos los bits (0->1 y 1->0)

    return [1 - x for x in a]

def negate_bits(a):

    # Juan: Se obtiene el negativo en C2 aplicando complemento a 1 y sumando 1 (~a + 1)

    return add_bits(invert_bits(a), int_to_bits(1))

def sub_bits(a, b):

    # Juan: Se hace resta como a + (-b) usando complemento a 2

    return add_bits(a, negate_bits(b))

def shift_left(a, k=1):

    # Juan: Se desplaza la lista de bits a la izquierda k veces, entran ceros por la derecha

    assert len(a) == BIT_WIDTH  # Juan: Se valida que haya 16 bits

    if k <= 0:  # Juan: Si k es 0 o negativo, no se modifica

        return a[:]
    k = min(k, BIT_WIDTH)  # Juan: Se limita k maximo a 16

    return a[k:] + [0] * k  # Juan: Se quitan los primeros k bits y se agregan ceros a la derecha

def shift_right(a, k=1):

    # Juan: Se desplaza la lista de bits a la derecha k veces, entran ceros por la izquierda

    assert len(a) == BIT_WIDTH

    if k <= 0:

        return a[:]

    k = min(k, BIT_WIDTH)

    return [0] * k + a[:BIT_WIDTH - k]  # Juan: Se agregan ceros al inicio y se recortan bits del final


def unsigned_ge(a, b):

    # Juan: Se compara si a >= b como enteros sin signo

    assert len(a) == BIT_WIDTH and len(b) == BIT_WIDTH

    for i in range(BIT_WIDTH):  # Juan: Se compara bit a bit de izquierda a derecha

        if a[i] != b[i]:  # Juan: Si hay diferencia, el mayor bit determina el mayor numero

            return a[i] > b[i]

    return True  # Juan: Si todos son iguales, a >= b es verdadero

def unsigned_sub(a, b):

    # Juan: Se realiza resta sin signo usando internamente C2

    return sub_bits(a, b)

def abs_bits(a):

    # Juan: Se obtiene el valor absoluto. Si es negativo se niega, si no se copia igual

    return negate_bits(a) if is_negative(a) else copy_bits(a)

def mul_bits(a, b):
    # Juan: Se realiza multiplicacion por desplazamiento y suma repetida. El signo se trata aparte

    assert len(a) == BIT_WIDTH and len(b) == BIT_WIDTH

    neg = (is_negative(a) != is_negative(b))  # Juan: Se determina si el resultado debe ser negativo

    x = abs_bits(a)  # Juan: Se toma valor absoluto de a

    y = abs_bits(b)  # Juan: Se toma valor absoluto de b

    res = int_to_bits(0)  # Juan: Se instala acumulador en 0

    for _ in range(BIT_WIDTH):  # Juan: Se itera 16 veces (por cada bit de y)

        if y[-1] == 1:  # Juan: Si el bit menos significativo de y es 1, se suma x al resultado

            res = add_bits(res, x)
        x = shift_left(x, 1)  # Juan: Se desplaza x a la izquierda (multiplica por 2)

        y = shift_right(y, 1)  # Juan: Se desplaza y a la derecha (divide por 2)

    if neg:  # Juan: Si el resultado debe ser negativo, se aplica negacion

        res = negate_bits(res)

    return res

def div_bits(a, b):

    # Juan: Se realiza division entera con truncamiento hacia 0. Si b=0, se lanza excepcion

    assert len(a) == BIT_WIDTH and len(b) == BIT_WIDTH

    if all(bit == 0 for bit in b):  # Juan: Se detecta si el divisor es 0

        raise ZeroDivisionError("Division por cero")

    neg = (is_negative(a) != is_negative(b))  # Juan: Se determina si el resultado debe ser negativo

    x = abs_bits(a)  # Juan: Se convierte dividendo a positivo

    y = abs_bits(b)  # Juan: Se convierte divisor a positivo

    quotient = [0] * BIT_WIDTH  # Juan: Se instala lista de bits para el cociente

    remainder = [0] * BIT_WIDTH  # Juan: Se instala lista de bits para el residuo

    for i in range(BIT_WIDTH):  # Juan: Se itera 16 veces por cada bit del dividendo

        next_bit = x[i]  # Juan: Se toma el siguiente bit del dividendo

        remainder = shift_left(remainder, 1)  # Juan: Se desplaza el residuo a la izquierda

        remainder[-1] = next_bit  # Juan: Se coloca el nuevo bit en el residuo

        if unsigned_ge(remainder, y):  # Juan: Si el residuo es mayor o igual al divisor

            remainder = unsigned_sub(remainder, y)  # Juan: Se resta el divisor al residuo

            quotient[i] = 1  # Juan: Se coloca 1 en el bit correspondiente del cociente

    if neg:  # Juan: Si el resultado debe ser negativo, se aplica negacion

        quotient = negate_bits(quotient)

    return quotient  # Juan: Se retorna el cociente final

def bits_to_str(bits, group=4):

    # Juan: Se convierte la lista de bits a texto agrupado, ej: "0000 1111 ..."

    s = ''.join(str(b) for b in bits)  # Juan: Se concatena cada bit como caracter

    return ' '.join(s[i:i+group] for i in range(0, len(s), group))

    # Juan: Se agrupa el texto cada 'group' bits separados por espacio

## Parser y evaluacion (tokenizador + Shunting Yard + RPN)

- Tokenizador: Un tokenizador es un componente que divide el texto de una expresión en unidades mínimas llamadas tokens
- Shunting Yard: El algoritmo Shunting Yard es una forma de ordenar los números y operadores de una expresión matemática para que una computadora pueda resolver fácilmente
- RPN: Una forma de escribir expresiones sin necesidad de paréntesis, donde los operadores van después de sus operandos

In [4]:
# Juan: Procesa la expresion y la evalua con el motor de 16 bits

# Juan: Maneja operadores + y - unarios correctamente (ej 5+5/2-8)

OPERATORS = {"+": 1, "-": 1, "*": 2, "/": 2}  # Juan: Se instala tabla con precedencias (suma/resta =1, mult/div =2)

def tokenize(expr: str):

    expr = expr.replace(" ", "")  # Juan: Se eliminan espacios de la expresion

    tokens = []  # Juan: Se instala lista vacia para los tokens

    i = 0  # Juan: Se instala indice de recorrido

    number = ""  # Juan: Se instala acumulador de digitos numericos

    prev = None  # Juan: Se instala registro del token previo

    def push_number():

        # Juan: Funcion interna que envia el numero acumulado a la lista de tokens

        nonlocal number, prev

        if number != "":  # Juan: Se valida que haya contenido numerico

            tokens.append(number)  # Juan: Se agrega el numero como token

            prev = number  # Juan: Se guarda el token previo

            number = ""  # Juan: Se limpia el acumulador

    while i < len(expr):  # Juan: Se recorre cada caracter de la expresion

        ch = expr[i]

        if ch.isdigit():  # Juan: Si es digito

            number += ch  # Juan: Se acumula en el numero actual

            i += 1  # Juan: Se avanza al siguiente caracter

        elif ch in "+-*/":  # Juan: Si es operador

            if number != "":  # Juan: Si hay numero pendiente

                push_number()  # Juan: Se envia a la lista

            if ch in "+-" and (prev is None or prev in OPERATORS):

                number = ch  # Juan: Se detecta signo unario y se inicia numero con signo

            else:

                tokens.append(ch)  # Juan: Se agrega el operador como token

                prev = ch  # Juan: Se actualiza el token previo

            i += 1

        else:

            raise ValueError(f"Caracter no valido: {ch!r}")  # Juan: Se lanza error si el caracter no es permitido

    push_number()  # Juan: Se envia ultimo numero pendiente

    return tokens  # Juan: Se retornan los tokens extraidos

def to_rpn(tokens):

    # Juan: Se convierte lista de tokens a notacion polaca inversa (RPN) usando algoritmo de la pila

    output, stack = [], []  # Juan: Se instalan pilas de salida y operadores

    for t in tokens:  # Juan: Se recorre cada token

        if t in OPERATORS:  # Juan: Si es operador

            while stack and stack[-1] in OPERATORS and OPERATORS[stack[-1]] >= OPERATORS[t]:

                # Juan: Se desapilan operadores con mayor o igual precedencia

                output.append(stack.pop())

            stack.append(t)  # Juan: Se apila el operador actual

        else:

            output.append(t)  # Juan: Si es numero, se envia directo a salida

    while stack:  # Juan: Se vacia la pila restante

        output.append(stack.pop())

    return output  # Juan: Se retorna la lista en RPN

def eval_rpn(rpn_tokens):

    # Juan: Se evalua expresion en notacion RPN usando el motor de 16 bits

    stack = []  # Juan: Se instala pila de operandos

    avisos = []  # Juan: Se instala lista para mensajes de advertencia

    for t in rpn_tokens:

        if t in OPERATORS:  # Juan: Si es operador

            b = stack.pop()  # Juan: Se desapila segundo operando

            a = stack.pop()  # Juan: Se desapila primer operando

            if t == "+":

                stack.append(add_bits(a, b))  # Juan: Se suma usando el motor de 16 bits

            elif t == "-":

                stack.append(sub_bits(a, b))  # Juan: Se resta usando el motor de 16 bits

            elif t == "*":

                stack.append(mul_bits(a, b))  # Juan: Se multiplica usando el motor de 16 bits

            elif t == "/":

                try:
                    stack.append(div_bits(a, b))  # Juan: Se divide usando el motor de 16 bits

                except ZeroDivisionError:

                    raise  # Juan: Se relanza la excepcion si hay division por cero

        else:

            n = int(t)  # Juan: Se convierte token a entero Python

            if n < -32768 or n > 32767:

                avisos.append(f"Literal {n} envuelto (mod 2^16)")

                # Juan: Se agrega advertencia si el literal excede el rango de 16 bits

            stack.append(int_to_bits(n))  # Juan: Se convierte el entero a bits C2 y se apila

    if len(stack) != 1:

        raise ValueError("Expresion mal formada")  # Juan: Se lanza error si la pila no queda con un solo resultado

    bits = stack[0]  # Juan: Se toma el resultado final en bits

    return bits, bits_to_int(bits), avisos  # Juan: Se retorna resultado en bits, en entero y avisos

def eval_expr(expr: str):

    # Juan: Se procesa toda la expresion en texto y se retorna estructura con todos los datos

    tokens = tokenize(expr)  # Juan: Se tokeniza la expresion

    rpn = to_rpn(tokens)  # Juan: Se convierte a notacion RPN

    bits, val, avisos = eval_rpn(rpn)  # Juan: Se evalua la expresion RPN

    return {

        "expr": expr,  # Juan: Se guarda la expresion original

        "tokens": tokens,  # Juan: Se guardan los tokens

        "rpn": rpn,  # Juan: Se guarda la RPN

        "bits": bits,  # Juan: Se guarda el resultado en bits

        "bits_str": bits_to_str(bits),  # Juan: Se guarda representacion de bits en texto

        "int": val,  # Juan: Se guarda el valor entero con signo

        "avisos": avisos  # Juan: Se guardan advertencias de rango

    }


## Pruebas basicas

In [5]:
# Juan: Verifica precedencia, negativos, truncamiento en division y overflow

tests = [

    ("4+5*2", 14),  # Juan: Se espera 4 + (5*2) = 14 por precedencia de * sobre +

    ("5+5/2-8", -1),  # Juan: Se espera (5 + (5/2 truncado a 2)) - 8 = -1

    ("-6/2", -3),  # Juan: Se espera -6 / 2 = -3 (negativo unario manejado)

    ("7*3-2", 19),  # Juan: Se espera (7*3) - 2 = 19

    ("10/3", 3),  # Juan: Se espera 10 / 3 = 3 (division truncada hacia 0)

    ("-10/3", -3),  # Juan: Se espera -10 / 3 = -3 (division truncada hacia 0)

    (
        "30000+30000",

        (30000+30000) & 0xFFFF if ((30000+30000)&0x8000)==0 else ((30000+30000)&0xFFFF)-0x10000

        # Juan: Se calcula resultado esperado aplicando overflow de 16 bits (C2)

    ),

]

for expr, expected in tests:  # Juan: Se recorre cada prueba

    out = eval_expr(expr)  # Juan: Se evalua la expresion con el motor de 16 bits

    print(f"{expr:>12} = {out['int']:>7}  | bits: {out['bits_str']}  | avisos: {out['avisos']}")

    # Juan: Se imprime la expresion, el resultado en decimal, los bits y los avisos generados

    assert out["int"] == expected, f"Fallo en {expr}: {out['int']} != {expected}"

    # Juan: Se verifica que el resultado obtenido sea igual al esperado, si no lanza error

print("\nListo: pruebas basicas OK")

# Juan: Se imprime mensaje de exito si todas las pruebas pasan


       4+5*2 =      14  | bits: 0000 0000 0000 1110  | avisos: []
     5+5/2-8 =      -1  | bits: 1111 1111 1111 1111  | avisos: []
        -6/2 =      -3  | bits: 1111 1111 1111 1101  | avisos: []
       7*3-2 =      19  | bits: 0000 0000 0001 0011  | avisos: []
        10/3 =       3  | bits: 0000 0000 0000 0011  | avisos: []
       -10/3 =      -3  | bits: 1111 1111 1111 1101  | avisos: []
 30000+30000 =   -5536  | bits: 1110 1010 0110 0000  | avisos: []

Listo: pruebas basicas OK


## Pruebas aleatorias tipo random

In [6]:

# Juan: Ensayos aleatorios sencillos

import random  # Juan: Se importa el modulo random para generar numeros aleatorios

def py_style_trunc_div(a, b):

    # Juan: Se implementa division con truncamiento hacia 0 como en Python para enteros

    if b == 0:  # Juan: Se verifica division por cero

        raise ZeroDivisionError  # Juan: Se lanza error si b es 0

    sign = -1 if (a < 0) ^ (b < 0) else 1

    # Juan: Se determina signo del resultado usando XOR (diferente signo => negativo)

    q = abs(a) // abs(b)  # Juan: Se calcula division entera usando valores absolutos

    return sign * q  # Juan: Se aplica el signo calculado al cociente

def wrap16(n):

    # Juan: Se ajusta el valor a 16 bits con signo (C2)

    n &= 0xFFFF  # Juan: Se aplica mascara de 16 bits para recortar el valor

    return n if n < 0x8000 else n - 0x10000

    # Juan: Si el bit de signo esta en 0, se retorna n; si esta en 1, se resta 2^16

random.seed(123)  # Juan: Se fija semilla para reproducibilidad de numeros aleatorios

for _ in range(60):  # Juan: Se ejecutan 60 pruebas aleatorias

    a = random.randint(-20000, 20000)  # Juan: Se genera entero aleatorio como primer operando

    b = random.randint(-20000, 20000) or 1

    # Juan: Se genera segundo operando aleatorio, si da 0 se cambia por 1 para evitar division por cero

    for op in ['+', '-', '*', '/']:  # Juan: Se prueba con los 4 operadores basicos

        expr = f"{a}{op}{b}"  # Juan: Se construye la expresion en texto

        out = eval_expr(expr)['int']  # Juan: Se evalua la expresion y se obtiene resultado entero

        if op == '+':

            exp = wrap16(a + b)  # Juan: Se calcula resultado esperado sumando y aplicando overflow

        elif op == '-':

            exp = wrap16(a - b)  # Juan: Se calcula resultado esperado restando y aplicando overflow

        elif op == '*':

            exp = wrap16(a * b)  # Juan: Se calcula resultado esperado multiplicando y aplicando overflow

        elif op == '/':

            exp = wrap16(py_style_trunc_div(a, b))

            # Juan: Se calcula resultado esperado usando division truncada y overflow

        assert out == exp, (expr, out, exp)

        # Juan: Se verifica que el resultado obtenido sea igual al esperado, si no lanza error

print("Listo: pruebas aleatorias OK")

# Juan: Se imprime mensaje de exito si todas las pruebas pasan correctamente


Listo: pruebas aleatorias OK


## UI Tkinter

In [7]:
# Juan: Interfaz

def run_app():

    import tkinter as tk  # Juan: Se importa tkinter como base para GUI

    from tkinter import ttk, messagebox  # Juan: Se importan componentes avanzados y mensajes

    COLORS = {

        "bg": "#0b1220",

        "panel": "#0f172a",

        "panel2": "#111827",

        "accent": "#2563eb",

        "accent2": "#1d4ed8",

        "text": "#e5e7eb",

        "muted": "#9ca3af",

    }  # Juan: Se instala paleta de colores para la interfaz

    root = tk.Tk()  # Juan: Se crea ventana principal

    root.title("Evaluador 16 bits (C2) — Proyecto 1")  # Juan: Se asigna titulo de ventana

    root.configure(bg=COLORS["bg"])  # Juan: Se aplica color de fondo

    # Centrar ventana

    def center(w=900, h=640):

        ws = root.winfo_screenwidth()  # Juan: Se obtiene ancho de pantalla

        hs = root.winfo_screenheight()  # Juan: Se obtiene alto de pantalla

        x = int((ws/2) - (w/2))  # Juan: Se calcula coordenada X centrada

        y = int((hs/2) - (h/2))  # Juan: Se calcula coordenada Y centrada

        root.geometry(f"{w}x{h}+{x}+{y}")  # Juan: Se aplica tamaño y posicion centrada

    center()  # Juan: Se centra al iniciar

    style = ttk.Style(root)  # Juan: Se crea objeto de estilo para widgets

    try:

        style.theme_use("clam")  # Juan: Se aplica tema 'clam'

    except Exception:

        pass  # Juan: Si no existe el tema, se ignora

    # Juan: Se configuran estilos visuales personalizados

    style.configure("Panel.TFrame", background=COLORS["panel"])

    style.configure("Panel2.TFrame", background=COLORS["panel2"])

    style.configure("Title.TLabel", background=COLORS["panel"], foreground=COLORS["text"], font=("Segoe UI", 16, "bold"))

    style.configure("Sub.TLabel", background=COLORS["panel"], foreground=COLORS["muted"], font=("Segoe UI", 10))

    style.configure("Mono.TLabel", background=COLORS["panel2"], foreground=COLORS["text"], font=("Consolas", 11))

    style.configure("H1.TLabel", background=COLORS["panel2"], foreground=COLORS["text"], font=("Segoe UI", 11, "bold"))

    style.configure("Btn.TButton", font=("Segoe UI", 10, "bold"), padding=8)


    # Juan: Se crea cabecera

    header = ttk.Frame(root, style="Panel.TFrame", padding=16)

    header.pack(fill="x")
    ttk.Label(header, text="🧮 Evaluador 16 bits — Complemento a 2", style="Title.TLabel").pack(anchor="w")

    ttk.Label(header, text="Suma, resta, multiplicacion y division (sin parentesis, con precedencia)", style="Sub.TLabel").pack(anchor="w", pady=(4,0))

    # Juan: Se crea cuerpo principal

    body = ttk.Frame(root, style="Panel.TFrame", padding=16)

    body.pack(fill="both", expand=True)

    # Juan: Se crea fila para la entrada de expresion

    entry_frame = ttk.Frame(body, style="Panel.TFrame")

    entry_frame.pack(fill="x", pady=(0, 10))

    ttk.Label(entry_frame, text="", style="H1.TLabel").grid(row=0, column=0, sticky="w", pady=(0, 6))

    expr_var = tk.StringVar(value="5+5/2-8")  # Juan: Se instala variable de texto

    entry = tk.Entry(entry_frame, textvariable=expr_var, font=("Consolas", 12), relief="flat",

                     bg=COLORS["panel2"], fg=COLORS["text"], insertbackground=COLORS["text"])

    entry.grid(row=1, column=0, sticky="ew")

    entry_frame.columnconfigure(0, weight=1)

    # Juan: Se crean botones rapidos de ejemplos

    btns = ttk.Frame(entry_frame, style="Panel.TFrame")

    btns.grid(row=1, column=1, padx=(10,0), sticky="n")

    def set_expr(s):

        expr_var.set(s)

        entry.icursor("end")

        entry.focus_set()

    for ex in ["4+5*2", "5+5/2-8", "-6/2", "7*3-2", "30000+30000"]:

        ttk.Button(btns, text=ex, style="Btn.TButton", command=lambda s=ex: set_expr(s)).pack(fill="x", pady=2)

    # Juan: Se crean botones Evaluar y Limpiar

    actions = ttk.Frame(body, style="Panel.TFrame")

    actions.pack(fill="x", pady=(6, 12))

    eval_btn = ttk.Button(actions, text="Evaluar", style="Btn.TButton")

    clear_btn = ttk.Button(actions, text="Limpiar", style="Btn.TButton")

    eval_btn.pack(side="left")

    clear_btn.pack(side="left", padx=(8,0))

    # Juan: Se crea grilla con tarjetas para mostrar resultados

    grid = ttk.Frame(body, style="Panel.TFrame")

    grid.pack(fill="both", expand=True)

    card1 = ttk.Frame(grid, style="Panel2.TFrame", padding=12)

    card2 = ttk.Frame(grid, style="Panel2.TFrame", padding=12)

    card3 = ttk.Frame(grid, style="Panel2.TFrame", padding=12)

    card4 = ttk.Frame(grid, style="Panel2.TFrame", padding=12)

    card5 = ttk.Frame(grid, style="Panel2.TFrame", padding=12)

    card1.grid(row=0, column=0, sticky="nsew", padx=6, pady=6)

    card2.grid(row=0, column=1, sticky="nsew", padx=6, pady=6)

    card3.grid(row=1, column=0, sticky="nsew", padx=6, pady=6)

    card4.grid(row=1, column=1, sticky="nsew", padx=6, pady=6)

    card5.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=6, pady=6)

    grid.columnconfigure(0, weight=1)

    grid.columnconfigure(1, weight=1)

    grid.rowconfigure(2, weight=1)

    # Juan: Se crean etiquetas para cada tarjeta

    ttk.Label(card1, text="Decimal", style="H1.TLabel").grid(row=0, column=0, sticky="w")

    dec_val = ttk.Label(card1, text="—", style="Mono.TLabel")

    dec_val.grid(row=1, column=0, sticky="w", pady=(6,0))

    ttk.Label(card2, text="Binario (16 bits, C2)", style="H1.TLabel").grid(row=0, column=0, sticky="w")

    bits_val = ttk.Label(card2, text="—", style="Mono.TLabel")

    bits_val.grid(row=1, column=0, sticky="w", pady=(6,0))

    ttk.Label(card3, text="Tokens", style="H1.TLabel").grid(row=0, column=0, sticky="w")

    tokens_val = ttk.Label(card3, text="—", style="Mono.TLabel")

    tokens_val.grid(row=1, column=0, sticky="w", pady=(6,0))

    ttk.Label(card4, text="RPN", style="H1.TLabel").grid(row=0, column=0, sticky="w")

    rpn_val = ttk.Label(card4, text="—", style="Mono.TLabel")

    rpn_val.grid(row=1, column=0, sticky="w", pady=(6,0))

    ttk.Label(card5, text="Avisos", style="H1.TLabel").grid(row=0, column=0, sticky="w")

    avisos_val = ttk.Label(card5, text="—", style="Mono.TLabel")

    avisos_val.grid(row=1, column=0, sticky="w", pady=(6,0))

    # Juan: Seccion de historial con scrollbar

    hist = ttk.Frame(body, style="Panel.TFrame")

    hist.pack(fill="both", expand=True, pady=(8,0))

    ttk.Label(hist, text="Historial", style="H1.TLabel").pack(anchor="w")

    container = ttk.Frame(hist, style="Panel.TFrame")

    container.pack(fill="both", expand=True)

    scroll = tk.Scrollbar(container, orient="vertical")

    h_list = tk.Listbox(container, activestyle="dotbox",

                        bg=COLORS["panel2"], fg=COLORS["text"],

                        selectbackground=COLORS["accent"], selectforeground="#ffffff",

                        font=("Consolas", 11), yscrollcommand=scroll.set, relief="flat")

    scroll.config(command=h_list.yview)

    h_list.pack(side="left", fill="both", expand=True)

    scroll.pack(side="right", fill="y")

    # Juan: Funciones principales

    def do_eval(evt=None):

        s = expr_var.get().strip()

        if not s:

            messagebox.showwarning("Aviso", "Escriba una expresion")

            return

        try:

            out = eval_expr(s)

            dec_val.config(text=str(out['int']))

            bits_val.config(text=str(out['bits_str']))

            tokens_val.config(text=str(out['tokens']))

            rpn_val.config(text=str(out['rpn']))

            aviso_txt = " · ".join(out["avisos"]) if out["avisos"] else "—"

            avisos_val.config(text=aviso_txt)

            h_list.insert("end", f"{s}  =>  {out['int']}  [{out['bits_str']}]")

            h_list.see("end")

        except ZeroDivisionError:

            messagebox.showerror("Error", "Division por cero")

        except Exception as e:

            messagebox.showerror("Error", str(e))

    def do_clear(evt=None):

        expr_var.set("")

        dec_val.config(text="—")

        bits_val.config(text="—")

        tokens_val.config(text="—")

        rpn_val.config(text="—")

        avisos_val.config(text="—")

        entry.focus_set()

    def on_history(evt):

        if not h_list.curselection():

            return

        line = h_list.get(h_list.curselection()[0])

        if "  =>  " in line:

            set_expr(line.split("  =>  ")[0])

    # Juan: Se configuran eventos y atajos

    h_list.bind("<<ListboxSelect>>", on_history)

    entry.bind("<Return>", do_eval)

    root.bind("<Control-l>", do_clear)

    eval_btn.config(command=do_eval)

    clear_btn.config(command=do_clear)

    root.minsize(880, 620)  # Juan: Se fija tamaño minimo de ventana

    root.mainloop()  # Juan: Se inicia bucle principal de la aplicacion


# Juan: Lanzamiento automatico con manejo de error si no hay entorno grafico

try:

    run_app()

except Exception as e:

    print("Juan: No se pudo abrir la UI de Tkinter. Si esta en un entorno sin interfaz grafica (por ejemplo, Colab), ejecute este cuaderno en su equipo local")

    print("Juan: Detalle:", e)


Juan: No se pudo abrir la UI de Tkinter. Si esta en un entorno sin interfaz grafica (por ejemplo, Colab), ejecute este cuaderno en su equipo local
Juan: Detalle: no display name and no $DISPLAY environment variable
